In [2]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight

#Reads CSV file based on given filename
def read_File(filename):
  file = pd.read_csv(filename[0],sep=' ',skiprows=1,header=None)
  file_loaded = file.iloc[:,[0,1,6]]
  return file_loaded

#Reads all the CSV files inside a folder
def send_Features(folderName):
  feature_List = []
  for ii in [1,2,4,5,6]:
    directory = folderName + '*' + str(ii) + '.svc'
    feature = read_File(glob.glob(directory))
    scaler = MinMaxScaler(feature_range=(0.00001,1))
    scaled_feature = scaler.fit_transform(feature)
    feature_List.append(scaled_feature)
  return feature_List

#Gets all the data from the folders
def get_Folder(foldername, file_count, file_start=[1,1]):
  feature_List = []
  for ii in range(0,len(foldername)):
    file_Directory = 'EMOTHAW/'+ foldername[ii]
    for iii in range(file_start[ii],file_count[ii]): 
      if(iii < 10):
        filename = file_Directory + '/user0000' + str(iii) + '/session00001/'
      else:
        filename = file_Directory + '/user000' + str(iii) + '/session00001/'
      feature_List.append(send_Features(filename))
  folder_num = 0
  for amount in range(len(file_count)):
    folder_num += file_count[amount] - file_start[amount]
  for ii in range(folder_num):   
    feature_List[ii] = keras.preprocessing.sequence.pad_sequences(feature_List[ii], padding='post', dtype='float32',maxlen=13000)
    feature_List[ii] = np.hstack(feature_List[ii])  
    feature_List[ii] = tf.squeeze(feature_List[ii]) 
  return (feature_List)
    
def get_Output():
  scores = np.array(dass_Scores.values)
  output = []
  for score in scores:
    if(score >= 21):
      output.append(2)
    elif(score >= 14): 
      output.append(2)
    elif(score >= 10):
      output.append(1)
    else:
      output.append(0)

  return np.array(output)

print("Import and Defining functions complete")


Import and Defining functions complete


In [3]:
#Gets input values and then prints shape
X = get_Folder(['Collection1','Collection2'],[46,85])
X = tf.reshape(X,[129,50,1,260,15])
print('The shape of X is:',X.shape)#X.shape)


#Gets output values and prints shape
dass_Scores = pd.read_csv('EMOTHAW/output.txt',sep=' ',header=None)
Y = tf.squeeze(dass_Scores)
print('The shape of Y is:',Y.shape)


The shape of X is: (129, 50, 1, 260, 15)
The shape of Y is: (129,)
The shape of Y is: (129, 1)
{0: 1.29, 1: 0.645, 2: 0.43, 3: 0.4690909090909091, 4: 0.3225, 5: 0.516, 6: 0.5733333333333334, 7: 0.516, 8: 0.5733333333333334, 9: 0.86, 10: 1.29, 11: 1.032, 12: 1.032, 13: 5.16, 14: 2.58, 15: 2.58, 16: 2.58, 17: 1.72, 18: 5.16, 19: 2.58, 20: 2.58, 21: 5.16, 22: 5.16, 23: 5.16, 24: 2.58}


In [4]:
model = keras.Sequential()
model.add(tf.keras.layers.Masking(mask_value=0, input_shape=(50,1,260,15)))
model.add(keras.layers.ConvLSTM2D(filters=30, kernel_size=(1,3), activation='relu',padding="same",return_sequences=True))
model.add(keras.layers.ConvLSTM2D(filters=15, kernel_size=(1,3), activation='relu',padding="same"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(25, activation='relu'))
model.add(keras.layers.Dense(1))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=700,
    decay_rate=0.5)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=opt, loss='mse', metrics=['acc'])
model.summary()
model.fit(X, Y, validation_split=0.25, epochs=60, batch_size=40, verbose=2)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 50, 1, 260, 15)    0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 50, 1, 260, 30)    16320     
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 1, 260, 15)        8160      
                                                                 
 flatten (Flatten)           (None, 3900)              0         
                                                                 
 dense (Dense)               (None, 200)               780200    
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 50)                5

KeyboardInterrupt: 

In [7]:
output = model.predict(X)
output = tf.squeeze(output)

adjusted = output
frame = pd.DataFrame({'Real':tf.squeeze(Y),'Predicted':output},index=range(129))
pd.set_option('display.max_rows', None)
frame


5/5 [==============================] - 3s 599ms/step


,Real,Predicted
0,2,6.664570
1,1,6.920356
2,5,7.284317
3,13,6.814342
4,5,7.747361
5,11,9.853558
6,8,7.137829
7,3,6.153368
8,8,7.089871
9,17,7.618398


In [6]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=3e-6,
    decay_steps=400,
    decay_rate=0.9)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

model.fit(X, Y, validation_split=0.25, epochs=50, batch_size=40, verbose=2)

Epoch 1/50
3/3 - 11s - loss: 34.1367 - acc: 0.0729 - val_loss: 39.2014 - val_acc: 0.0303 - 11s/epoch - 4s/step
Epoch 2/50
3/3 - 11s - loss: 34.2288 - acc: 0.0729 - val_loss: 39.3718 - val_acc: 0.0303 - 11s/epoch - 4s/step
Epoch 3/50
3/3 - 12s - loss: 34.0713 - acc: 0.0729 - val_loss: 39.4194 - val_acc: 0.0303 - 12s/epoch - 4s/step
Epoch 4/50
3/3 - 12s - loss: 34.0373 - acc: 0.0729 - val_loss: 39.3603 - val_acc: 0.0303 - 12s/epoch - 4s/step
Epoch 5/50
3/3 - 12s - loss: 33.9081 - acc: 0.0729 - val_loss: 39.2189 - val_acc: 0.0303 - 12s/epoch - 4s/step
Epoch 6/50
3/3 - 12s - loss: 33.8425 - acc: 0.0729 - val_loss: 39.0522 - val_acc: 0.0303 - 12s/epoch - 4s/step
Epoch 7/50
3/3 - 12s - loss: 33.8314 - acc: 0.0729 - val_loss: 39.0348 - val_acc: 0.0303 - 12s/epoch - 4s/step
Epoch 8/50
3/3 - 12s - loss: 33.6505 - acc: 0.0729 - val_loss: 39.0924 - val_acc: 0.0303 - 12s/epoch - 4s/step
Epoch 9/50
3/3 - 12s - loss: 33.5723 - acc: 0.0729 - val_loss: 39.2571 - val_acc: 0.0303 - 12s/epoch - 4s/step
E